In [185]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Set pandas to display numbers with commas
pd.options.display.float_format = '{:,}'.format

## Import and Clean Data

In [186]:
# import and clean data

pluto_df = pd.read_csv("pluto.csv")
pluto_df.sort_values(by="zipcode", inplace=True)
pluto_df = pluto_df.reset_index(drop=True).copy()
pluto_df.dropna(inplace=True)
pluto_df.head()

,zipcode,borough,borocode,landuse,bldgarea
0,"10,001.0",MN,1.0,5.0,"611,625.0"
1,"10,001.0",MN,1.0,4.0,"13,489.0"
2,"10,001.0",MN,1.0,5.0,"34,000.0"
3,"10,001.0",MN,1.0,11.0,0.0
4,"10,001.0",MN,1.0,5.0,"2,008.0"


In [187]:
#One hot encoding for landuse

dummy_df = pd.get_dummies(pluto_df.landuse)
pluto_df.drop("landuse", axis=1, inplace=True)
data_df = pluto_df.join(dummy_df)
data_df.head()

,zipcode,borough,borocode,bldgarea,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,"10,001.0",MN,1.0,"611,625.0",0,0,0,0,1,0,0,0,0,0,0
1,"10,001.0",MN,1.0,"13,489.0",0,0,0,1,0,0,0,0,0,0,0
2,"10,001.0",MN,1.0,"34,000.0",0,0,0,0,1,0,0,0,0,0,0
3,"10,001.0",MN,1.0,0.0,0,0,0,0,0,0,0,0,0,0,1
4,"10,001.0",MN,1.0,"2,008.0",0,0,0,0,1,0,0,0,0,0,0


In [188]:
# Associate bldgarea with landuse values

cols = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]
for col in cols:
    data_df[col] = data_df[col] * data_df.bldgarea
data_df.drop("bldgarea", axis=1, inplace=True)
data_df.head()

,zipcode,borough,borocode,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,"10,001.0",MN,1.0,0.0,0.0,0.0,0.0,"611,625.0",0.0,0.0,0.0,0.0,0.0,0.0
1,"10,001.0",MN,1.0,0.0,0.0,0.0,"13,489.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"10,001.0",MN,1.0,0.0,0.0,0.0,0.0,"34,000.0",0.0,0.0,0.0,0.0,0.0,0.0
3,"10,001.0",MN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"10,001.0",MN,1.0,0.0,0.0,0.0,0.0,"2,008.0",0.0,0.0,0.0,0.0,0.0,0.0


In [189]:
# Create summary data

data_df.drop("borough", axis=1, inplace=True)
landuse_df = data_df.groupby("zipcode").agg({'borocode':'median',
                                                1.0:'sum',
                                                2.0:'sum',
                                                3.0:'sum',
                                                4.0:'sum', 
                                                5.0:'sum',
                                                6.0:'sum',
                                                7.0:'sum',
                                                8.0:'sum',
                                                9.0:'sum',
                                                10.0:'sum',
                                                11.0:'sum'})
landuse_df.reset_index(inplace=True)
landuse_df.head()

,zipcode,borocode,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,"10,001.0",1.0,"33,606.0","592,289.0","8,350,224.0","13,875,477.0","53,005,101.0","4,445,633.0","1,182,594.0","10,895,555.0","1,152,624.0","616,150.0",0.0
1,"10,002.0",1.0,"51,117.0","1,938,379.0","16,618,352.0","16,356,822.0","3,200,967.0","1,182,700.0","590,077.0","4,618,645.0","87,067.0","252,263.0",0.0
2,"10,003.0",1.0,"515,395.0","3,786,590.0","12,746,613.0","15,687,708.0","10,760,299.0","714,504.0","3,459.0","9,501,989.0","126,922.0","159,151.0",0.0
3,"10,004.0",1.0,0.0,0.0,"725,023.0","4,194,913.0","21,117,265.0","85,472.0","462,543.0","3,722,844.0","945,425.0","87,775.0",0.0
4,"10,005.0",1.0,0.0,0.0,"2,480,663.0","5,683,041.0","16,515,733.0",0.0,"682,450.0","43,568.0",0.0,0.0,0.0


In [190]:
# Relabel borocode so easy to remember (had to drop above b/c groupby only accepts numeric columns)

boro_dict = {"MN": 1.0, 
             "BX": 2.0, 
             "BK": 3.0, 
             "QN": 4.0, 
             "SI": 5.0}

def relabel(num):
    boros = list(boro_dict.keys())
    values = list(boro_dict.values())    
    return boros[values.index(num)]

landuse_df['borocode'] = landuse_df['borocode'].apply(relabel)

In [191]:
cols_dict = {1.0: "1,2 Family Resi",
             2.0: "Multi-Fam Walk-up",
             3.0: "Multi-Fam Elevator",
             4.0: "Mixed Resi/Comm",
             5.0: "Comm & Office",
             6.0: "Industrial",
             7.0: "Transport",
             8.0: "Public",
             9.0: "Open Space",
             10.0: "Parking",
             11.0: "Vacant"}

In [192]:
landuse_df.rename(columns = cols_dict, inplace=True)
landuse_df

,zipcode,borocode,"1,2 Family Resi",Multi-Fam Walk-up,Multi-Fam Elevator,Mixed Resi/Comm,Comm & Office,Industrial,Transport,Public,Open Space,Parking,Vacant
0,"10,001.0",MN,"33,606.0","592,289.0","8,350,224.0","13,875,477.0","53,005,101.0","4,445,633.0","1,182,594.0","10,895,555.0","1,152,624.0","616,150.0",0.0
1,"10,002.0",MN,"51,117.0","1,938,379.0","16,618,352.0","16,356,822.0","3,200,967.0","1,182,700.0","590,077.0","4,618,645.0","87,067.0","252,263.0",0.0
2,"10,003.0",MN,"515,395.0","3,786,590.0","12,746,613.0","15,687,708.0","10,760,299.0","714,504.0","3,459.0","9,501,989.0","126,922.0","159,151.0",0.0
3,"10,004.0",MN,0.0,0.0,"725,023.0","4,194,913.0","21,117,265.0","85,472.0","462,543.0","3,722,844.0","945,425.0","87,775.0",0.0
4,"10,005.0",MN,0.0,0.0,"2,480,663.0","5,683,041.0","16,515,733.0",0.0,"682,450.0","43,568.0",0.0,0.0,0.0
5,"10,006.0",MN,0.0,0.0,"2,406,484.0","1,409,933.0","7,224,839.0",0.0,0.0,"237,019.0",0.0,"305,026.0",0.0
6,"10,007.0",MN,"11,515.0","135,365.0","25,180,716.0","7,113,948.0","26,421,102.0","186,711.0","6,565.0","1,964,664.0",0.0,0.0,0.0
7,"10,009.0",MN,"49,533.0","4,551,976.0","7,867,831.0","16,437,543.0","1,094,334.0","132,152.0","175,000.0","1,778,954.0","52,703.0","3,358.0",0.0
8,"10,010.0",MN,"35,514.0","671,530.0","7,188,028.0","14,478,911.0","16,348,068.0","316,637.0","65,745.0","5,288,022.0","20,206.0","84,013.0",0.0
9,"10,011.0",MN,"1,029,569.0","4,471,226.0","14,800,035.0","18,123,314.0","15,296,761.0","1,648,286.0","464,332.0","4,294,146.0","12,323.0","78,016.0",0.0


## Find Nearest Neighbor

In [193]:
# Split DFs so that selected outer-borough zipcode can find nearest neighbor zip in Manhattan
manhattan_df = landuse_df[landuse_df.borocode == "MN"]
outer_boro_df = landuse_df[landuse_df.borocode != "MN"]

In [194]:
# Specify target zipcode

#zipcode = 11101 #Long Island City, QNS
#zipcode = 11104 #Sunnyside, QNS
#zipcode = 11355 #Flushing, QNS
#zipcode = 11201 #Brooklyn Heights, BK
zipcode = 10304 #Todt Hill, SI

In [195]:
# Pull out target zipcode from outer_boro_df
query_zip = outer_boro_df[outer_boro_df.zipcode == zipcode]

# Combine with Manhattan df to train model
train_data = pd.concat([query_zip, manhattan_df])

# Drop column with strings and reset index
train_data.drop("borocode", axis=1, inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [196]:
# Train Nearest Neighbors model

nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(train_data)
distances, indices = nbrs.kneighbors(train_data)

In [197]:
nearest_neighbor_index = indices[0][1]

In [198]:
train_data.iloc[nearest_neighbor_index, :]

zipcode                 10,030.0
1,2 Family Resi        774,088.0
Multi-Fam Walk-up    4,628,963.0
Multi-Fam Elevator   3,004,654.0
Mixed Resi/Comm      4,303,357.0
Comm & Office          110,686.0
Industrial                   0.0
Transport                    0.0
Public               1,341,992.0
Open Space               1,410.0
Parking                 56,356.0
Vacant                       0.0
Name: 28, dtype: float64

In [199]:
query_zip

,zipcode,borocode,"1,2 Family Resi",Multi-Fam Walk-up,Multi-Fam Elevator,Mixed Resi/Comm,Comm & Office,Industrial,Transport,Public,Open Space,Parking,Vacant
72,"10,304.0",SI,"14,242,428.0","1,899,750.0","3,222,559.0","1,417,842.0","1,085,077.0","435,189.0","70,512.0","2,456,708.0","65,787.0","29,529.0",0.0
